In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import copy
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import torch
import tqdm.notebook as tqdm

In [2]:
device = "cuda:0"
random_seed = 777
train_val_ratio = 0.8

In [3]:
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# !unzip ./dataset/dataset.zip  -d ./dataset

In [5]:
entire_train_dataset = datasets.ImageFolder(
    root="./dataset/casting_data/data/train",
    transform=transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor()
    ])
)

train_dataset_size = int(len(entire_train_dataset) * train_val_ratio)
val_dataset_size = len(entire_train_dataset) - train_dataset_size

train_dataset, val_dataset = random_split(entire_train_dataset, [train_dataset_size, val_dataset_size])

test_dataset = datasets.ImageFolder(
    root="./dataset/casting_data/data/test",
    transform=transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor()
    ])
)

print(f"Size - train : {len(train_dataset)}, val : {len(val_dataset)}, test : {len(test_dataset)}")

Size - train : 531, val : 133, test : 71


In [7]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=32,
    shuffle=True
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False
)

In [8]:
class InspectionModel(nn.Module):
    
    def __init__(self):
        super(InspectionModel, self).__init__()
        
        self.conv_1 = nn.Sequential(
            nn.Conv2d(1, 8, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_2 = nn.Sequential(
            nn.Conv2d(8, 16, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_3 = nn.Sequential(
            nn.Conv2d(16, 16, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(19600, 224),
            nn.ReLU(inplace=True),
            nn.Linear(224, 1)
        )
        
        
    def forward(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = x.view(x.size()[0], -1)
        
#         conv_output_size = x.size()[-1]
#         print(conv_output_size)
        
        x = self.fc(x)
        x = torch.sigmoid(x)
        
        return x

In [9]:
def evaluate(model, data_loader, loss_fn, hypothesis_threshold, device):
    model.eval()
    
    total_loss = 0
    n_batch = 0
    
    label_list = list()
    pred_list = list()
    
    for i, (img_batch, label_batch) in enumerate(data_loader):
        img_batch = img_batch.to(device)
        label_batch = label_batch.float().unsqueeze(dim=1).to(device)
        
        with torch.no_grad():
            hypothesis = model(img_batch)
            loss = loss_fn(hypothesis, label_batch)
        
        pred = (hypothesis > hypothesis_threshold).float()
        
        label_list.append(label_batch)
        pred_list.append(pred)
        total_loss += loss.item()
        n_batch += 1
    
    label_list = torch.cat(label_list).cpu().numpy()
    pred_list = torch.cat(pred_list).cpu().numpy()
    current_loss = total_loss / n_batch
    
    return dict(
        loss=current_loss,
        accuracy=accuracy_score(y_true=label_list, y_pred=pred_list),
        precision=precision_score(y_true=label_list, y_pred=pred_list),
        recall=recall_score(y_true=label_list, y_pred=pred_list),
        f1=f1_score(y_true=label_list, y_pred=pred_list)
    )

In [10]:
def train(model, optimizer_cls, loss_fn, n_epoch, lr, hypothesis_threshold, weight_decay, device, save_dir_path):
    model.train()
    optimizer = optimizer_cls(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    best_acc_epoch = 0
    best_f1_epoch = 0
    best_acc = 0
    best_f1 = 0
    best_model = None
    
    train_result_dict_list = list()
    val_result_dict_list = list()
    
    for epoch in tqdm.tqdm(range(n_epoch)):
        total_loss = 0
        n_batch = 0
        
        label_list = list()
        pred_list = list()
        
        for i, (img_batch, label_batch) in enumerate(train_loader):
            img_batch = img_batch.to(device)
            label_batch = label_batch.float().unsqueeze(dim=1).to(device)
            
            # Optimization
            optimizer.zero_grad()
            hypothesis = model(img_batch)
            loss = loss_fn(hypothesis, label_batch)
            loss.backward()
            optimizer.step()
            
            pred = (hypothesis > hypothesis_threshold).float()
            
            label_list.append(label_batch)
            pred_list.append(pred)
            total_loss += loss.item()
            n_batch += 1
        
        label_list = torch.cat(label_list).cpu().numpy()
        pred_list = torch.cat(pred_list).cpu().numpy()
        current_loss = total_loss / n_batch
        
        train_result_dict = dict(
            loss=current_loss,
            accuracy=accuracy_score(y_true=label_list, y_pred=pred_list),
            precision=precision_score(y_true=label_list, y_pred=pred_list),
            recall=recall_score(y_true=label_list, y_pred=pred_list),
            f1=f1_score(y_true=label_list, y_pred=pred_list)
        )
        val_result_dict = evaluate(
            model=model,
            data_loader=val_loader,
            loss_fn=loss_fn,
            hypothesis_threshold=hypothesis_threshold,
            device=device
        )
        
        if best_acc < val_result_dict['accuracy']:
            best_acc = val_result_dict['accuracy']
            best_acc_epoch = epoch
            best_model = copy.deepcopy(model)
        
        if best_f1 < val_result_dict['f1']:
            best_f1 = val_result_dict['f1']
            best_f1_epoch = epoch
        
        print(f"[Epoch - {epoch}]")
        print(f"Train - Accuracy : {round(train_result_dict['accuracy'], 7)}, Precision : {round(train_result_dict['precision'], 7)}, Recall : {round(train_result_dict['recall'], 7)}, F1 : {round(train_result_dict['f1'], 7)}")
        print(f"Val - Accuracy : {round(val_result_dict['accuracy'], 7)}, Precision : {round(val_result_dict['precision'], 7)}, Recall : {round(val_result_dict['recall'], 7)}, F1 : {round(val_result_dict['f1'], 7)}")
        print(f"Best Accuracy : {best_acc} (epoch : {best_acc_epoch})")
        print(f"Best F1 : {best_f1} (epoch : {best_f1_epoch})")
        
        train_result_dict_list.append(train_result_dict)
        val_result_dict_list.append(val_result_dict)
    
    entire_record_dict = dict(
        train_result_dict_list=train_result_dict_list,
        val_result_dict_list=val_result_dict_list
    )
    
    test_result_dict = evaluate(
        model=model,
        data_loader=test_loader,
        loss_fn=loss_fn,
        hypothesis_threshold=hypothesis_threshold,
        device=device
    )
    print("Test result :")
    print(test_result_dict)
    
    # Save the best model
    model_save_file_path = os.path.join(save_dir_path, f"original_best_model.pt")
    torch.save(best_model.state_dict(), model_save_file_path)
    print(f"Complete to save the model, path : {model_save_file_path}")
    
    # Save entire_record_dict
    record_save_file_path = os.path.join(save_dir_path, f"entire_record_dict.pkl")
    with open(record_save_file_path, "wb") as f:
        pickle.dump(entire_record_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Complete to save the record, path : {save_dir_path}")
        
    return entire_record_dict

In [11]:
model = InspectionModel()
model.to(device)

InspectionModel(
  (conv_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_3): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=19600, out_features=224, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=224, out_features=1, bias=True)
  )
)

In [12]:
entire_record_dict = train(
    model=model,
    optimizer_cls=optim.Adam,
    loss_fn=F.binary_cross_entropy,
    n_epoch=50,
    lr=1e-3,
    hypothesis_threshold=0.5,
    weight_decay=0,
    device=device,
    save_dir_path="./model"
)

  0%|          | 0/50 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Epoch - 0]
Train - Accuracy : 0.5160075, Precision : 0.4153005, Recall : 0.3362832, F1 : 0.3716381
Val - Accuracy : 0.5338346, Precision : 0.0, Recall : 0.0, F1 : 0.0
Best Accuracy : 0.5338345864661654 (epoch : 0)
Best F1 : 0 (epoch : 0)


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Epoch - 1]
Train - Accuracy : 0.5743879, Precision : 0.0, Recall : 0.0, F1 : 0.0
Val - Accuracy : 0.5338346, Precision : 0.0, Recall : 0.0, F1 : 0.0
Best Accuracy : 0.5338345864661654 (epoch : 0)
Best F1 : 0 (epoch : 0)
[Epoch - 2]
Train - Accuracy : 0.5894539, Precision : 0.7857143, Recall : 0.0486726, F1 : 0.0916667
Val - Accuracy : 0.6090226, Precision : 1.0, Recall : 0.1612903, F1 : 0.2777778
Best Accuracy : 0.6090225563909775 (epoch : 2)
Best F1 : 0.27777777777777773 (epoch : 2)
[Epoch - 3]
Train - Accuracy : 0.6516008, Precision : 0.6694215, Recall : 0.3584071, F1 : 0.4668588
Val - Accuracy : 0.6541353, Precision : 0.9444444, Recall : 0.2741935, F1 : 0.425
Best Accuracy : 0.6541353383458647 (epoch : 3)
Best F1 : 0.425 (epoch : 3)
[Epoch - 4]
Train - Accuracy : 0.787194, Precision : 0.7783251, Recall : 0.699115, F1 : 0.7365967
Val - Accuracy : 0.7969925, Precision : 0.739726, Recall : 0.8709677, F1 : 0.8
Best Accuracy : 0.7969924812030075 (epoch : 4)
Best F1 : 0.8 (epoch : 4)
[Ep

[Epoch - 32]
Train - Accuracy : 0.992467, Precision : 0.9955357, Recall : 0.9867257, F1 : 0.9911111
Val - Accuracy : 0.9398496, Precision : 0.9354839, Recall : 0.9354839, F1 : 0.9354839
Best Accuracy : 0.9398496240601504 (epoch : 32)
Best F1 : 0.9354838709677419 (epoch : 32)
[Epoch - 33]
Train - Accuracy : 0.9962335, Precision : 0.9955752, Recall : 0.9955752, F1 : 0.9955752
Val - Accuracy : 0.9323308, Precision : 0.9344262, Recall : 0.9193548, F1 : 0.9268293
Best Accuracy : 0.9398496240601504 (epoch : 32)
Best F1 : 0.9354838709677419 (epoch : 32)
[Epoch - 34]
Train - Accuracy : 0.9943503, Precision : 0.9955556, Recall : 0.9911504, F1 : 0.9933481
Val - Accuracy : 0.9172932, Precision : 0.9473684, Recall : 0.8709677, F1 : 0.907563
Best Accuracy : 0.9398496240601504 (epoch : 32)
Best F1 : 0.9354838709677419 (epoch : 32)
[Epoch - 35]
Train - Accuracy : 0.9981168, Precision : 1.0, Recall : 0.9955752, F1 : 0.9977827
Val - Accuracy : 0.9398496, Precision : 0.921875, Recall : 0.9516129, F1 : 0